In [ ]:
%pylab inline

from __future__ import print_function
import numpy as np
import theano
import theano.tensor as T
import time
import datetime
import os

import lasagne
from copy import deepcopy

import SimpleITK as sitk
import random
from skimage import exposure
from skimage.morphology import binary_closing

import sklearn.cross_validation

In [ ]:
def extract_patches(mri, label, half_patch_size=5, negative_subsample_ratio=500):
    
    assert mri.shape == label.shape
    
    z_size, x_size, y_size = mri.shape
    positive_patches = []
    negative_patches = []

    for z in xrange(half_patch_size, z_size-half_patch_size):
        if label[z,:,:].sum() > 1000: continue # workaround to remove white slices into the label dataset
        for x in xrange(half_patch_size, x_size-half_patch_size):
            for y in xrange(half_patch_size, y_size-half_patch_size):
                if label[z,x,y] == 1:
                    positive_patches.append(mri[z-half_patch_size:z+half_patch_size+1,x-half_patch_size:x+half_patch_size+1,y-half_patch_size:y+half_patch_size+1])
                elif label[z,x,y] == 0:
                    negative_patches.append(mri[z-half_patch_size:z+half_patch_size+1,x-half_patch_size:x+half_patch_size+1,y-half_patch_size:y+half_patch_size+1])
    
    random.shuffle(negative_patches)
    number_of_negative_cases = int(len(negative_patches) / float(negative_subsample_ratio))
    selected_negative_patches = deepcopy(negative_patches[:number_of_negative_cases])
    del(negative_patches)
    
    return (positive_patches, selected_negative_patches)

In [ ]:
def patchimg2differentview(patch):
    
    z_size, x_size, y_size = patch.shape
    
    X = np.zeros((z_size * 3, x_size, y_size), dtype=np.float32)
    
    counter = 0
    for z in xrange(z_size):
        X[counter,:,:] = patch[z,:,:]
        counter += 1
    for x in xrange(x_size):
        X[counter,:,:] = patch[:,x,:]
        counter += 1
    for y in xrange(y_size):
        X[counter,:,:] = patch[:,:,y]
        counter += 1
    
    return X.astype(np.float32)

In [ ]:
def patches2CNNformat(patches, label, half_patch_size=5):
    
    X = np.zeros((len(patches), 3 * ((2 * half_patch_size)+1), (half_patch_size*2)+1, (half_patch_size*2)+1), dtype=np.float32)
    y = np.zeros((len(patches)), dtype=np.int32) * -1
    
    for i, patch in enumerate(patches):
        X[i,:,:,:] = patchimg2differentview(patch)
        y[i] = label
    
    assert -1 not in y
    
    return X.astype(np.float32), y.astype(np.int32)

In [ ]:
def get_resized_img(img_fn, data_type = sitk.sitkFloat32):
    
    img = sitk.ReadImage(img_fn)
    size = img.GetSize()
    ratio = [1.0/i for i in img.GetSpacing()]
    new_size = [int(size[i]/ratio[i]) for i in range(3)]
    
    rimage = sitk.Image(new_size, data_type)
    rimage.SetSpacing((1,1,1))
    rimage.SetOrigin(img.GetOrigin())
    tx = sitk.Transform()
    
    interp = sitk.sitkLinear
    if data_type == sitk.sitkInt16:
        interp = sitk.sitkNearestNeighbor
    
    new_image = sitk.Resample(img, rimage, tx, interp, data_type)
    
    return sitk.GetArrayFromImage(new_image)

In [ ]:
def needles2tips(only_needles, image_array, number_of_slices=3):
    needles_masks_array = np.zeros_like(image_array).astype(float)  
    for file_item in only_needles:
        this_mask = file_item.astype(np.float)
        if np.sum(this_mask) < (np.shape(this_mask)[0] * np.shape(this_mask)[1] * np.shape(this_mask)[2]):
            this_mask = binary_closing(this_mask,selem=np.ones((3,3,3)))
            found=False
            row = np.shape(this_mask)[0]-1
            while found==False and row > 0: #< np.shape(this_mask)[0]-1 :
                #print(row)
                this_row = this_mask[row,:,:]
                if np.sum(this_row) > 0:
                    #print(row)
                    found = True
                    temp = np.add(needles_masks_array[row:row+1+number_of_slices,:,:],this_mask[row:row+1+number_of_slices,:,:])
                    temp[temp!=0] = 1
                    needles_masks_array[row:row+1+number_of_slices,:,:] = temp 
                row -= 1
    return needles_masks_array.astype(np.int32)

In [ ]:
def needles2tips_PAOLO(needles, mri, number_of_slices=3):
    tips = np.zeros_like(mri).astype(np.int32)  
    for needle in needles:
        needle = needle.astype(np.int32)
        if np.sum(needle) < (np.shape(needle)[0] * np.shape(needle)[1] * np.shape(needle)[2]):
            needle = binary_closing(needle, selem=np.ones((3,3,3)))
            for z in range(np.shape(mri)[0]-1, 0, -1):
                if np.sum(needle[z,:,:]) > 0.5 and z-number_of_slices-1 >= 0:
                    tmp = deepcopy(needle)
                    tmp[:z-number_of_slices-1,:,:] = 0
                    tips[tmp!=0] =1
                    del(tmp)
                    break
    
    tips[tips!=0]=1
    
    return tips.astype(np.int32)

In [ ]:
def pad_volume(img, half_patch_size=5):
    npad = ((half_patch_size,half_patch_size),(half_patch_size,half_patch_size),(half_patch_size,half_patch_size))
    return np.lib.pad(img, npad, "constant", constant_values=0)

In [ ]:
def data_for_CNN(general_folder, half_patch_size=5):
    folders_cases = os.listdir(general_folder)
    
    X_pos = np.ones((1, 3 * ((2 * half_patch_size)+1), (half_patch_size*2)+1, (half_patch_size*2)+1), dtype=np.float32) * -1.0
    y_pos = np.ones((1), dtype=np.int32) * -1.0
    
    X_neg = np.ones((1, 3 * ((2 * half_patch_size)+1), (half_patch_size*2)+1, (half_patch_size*2)+1), dtype=np.float32) * -1.0
    y_neg = np.ones((1), dtype=np.int32) * -1.0
    
    for folder_case in folders_cases:
        print("Patient #%s" % (folder_case))
        full_case_path = general_folder + os.sep + folder_case
        
        volumetric_files = os.listdir(full_case_path)
        
        assert "case.nrrd" in volumetric_files
        del(volumetric_files[volumetric_files.index("case.nrrd")])
        print(" %d needles file" %  (len(volumetric_files)))

        
        mri = get_resized_img(full_case_path + os.sep + "case.nrrd")
        unpad_mri = deepcopy(mri)
        #print("MRI min %f -- max %f" % (mri.min(), mri.max()))
        mri = pad_volume(mri, half_patch_size=half_patch_size)
        
        needles = []
        for volumetric_file in volumetric_files:
            label = get_resized_img(full_case_path + os.sep + volumetric_file, sitk.sitkInt16)
            label[label!=0]=1
            #if 1 in label: print("1!")
            needles.append(label)
        
        tips = needles2tips_PAOLO(needles, unpad_mri)
        #print("Tips min %f -- max %f" % (tips.min(), tips.max()))
        tips = pad_volume(tips, half_patch_size=half_patch_size)
        #tips[tips!=0]=1
        #tips = tips.astype(np.int32)
        positive_patches, negative_patches = extract_patches(mri, tips, half_patch_size=half_patch_size)
        if len(positive_patches) > 0:
            print(" %d positive patches and %d negative patches" % (len(positive_patches), len(negative_patches)))
            X_temp_pos, y_temp_pos = patches2CNNformat(positive_patches, 1, half_patch_size=half_patch_size)
            X_temp_neg, y_temp_neg = patches2CNNformat(negative_patches, 0, half_patch_size=half_patch_size)
            
            X_pos = np.concatenate((X_pos, X_temp_pos), axis=0)
            y_pos = np.concatenate((y_pos, y_temp_pos), axis=0)
            X_neg = np.concatenate((X_neg, X_temp_neg), axis=0)
            y_neg = np.concatenate((y_neg, y_temp_neg), axis=0)
    
    assert -1 in X_pos[0,:,:,:] and y_pos[0] == -1
    X_pos = X_pos[1:,:,:,:].astype(np.float32)
    y_pos = y_pos[1:].astype(np.int32)
    
    assert -1 in X_neg[0,:,:,:] and y_neg[0] == -1
    X_neg = X_neg[1:,:,:,:].astype(np.float32)
    y_neg = y_neg[1:].astype(np.int32)
    
    return X_pos, y_pos, X_neg, y_neg

In [ ]:
def join_Xy_posneg(X_pos, y_pos, X_neg, y_neg, test_size1=0.20, test_size2=0.33):
    
    X_pos_train, X_pos_test, y_pos_train, y_pos_test = sklearn.cross_validation.train_test_split(X_pos, y_pos, test_size=test_size1)
    X_pos_test, X_pos_val, y_pos_test, y_pos_val = sklearn.cross_validation.train_test_split(X_pos_test, y_pos_test, test_size=test_size2)

    X_neg_train, X_neg_test, y_neg_train, y_neg_test = sklearn.cross_validation.train_test_split(X_neg, y_neg, test_size=test_size1)
    X_neg_test, X_neg_val, y_neg_test, y_neg_val = sklearn.cross_validation.train_test_split(X_neg_test, y_neg_test, test_size=test_size2)

    X_train = np.concatenate((X_neg_train, X_pos_train), axis=0).astype(np.float32)
    y_train = np.concatenate((y_neg_train, y_pos_train), axis=0).astype(np.int32)

    X_val = np.concatenate((X_neg_val, X_pos_val), axis=0).astype(np.float32)
    y_val = np.concatenate((y_neg_val, y_pos_val), axis=0).astype(np.int32)

    X_test = np.concatenate((X_neg_test, X_pos_test), axis=0).astype(np.float32)
    y_test = np.concatenate((y_neg_test, y_pos_test), axis=0).astype(np.int32)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
half_patch_size = 5

In [ ]:
X_pos, y_pos, X_neg, y_neg = data_for_CNN("../LabelMaps", half_patch_size=half_patch_size)
#saved_data = np.load('../patches_jun27_halfpatchsize5odd.npz')
#X, y = saved_data['arr_0'], saved_data['arr_1']

In [ ]:
#np.savez('../patches_jul1_halfpatchsize5odd.npz', X_pos, y_pos, X_neg, y_neg)

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = join_Xy_posneg(X_pos, y_pos, X_neg, y_neg, test_size1=0.20, test_size2=0.33)

In [ ]:
m, s = X_train.mean(), X_train.std()

X_train -= m
X_train /= s

X_val -= m
X_val /= s

X_test -= m
X_test /= s

In [ ]:
X_train.shape, X_test.shape, X_val.shape

In [ ]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
def build_cnn(single_entry_shape, input_var=None):
    
    network = lasagne.layers.InputLayer(shape=(None, single_entry_shape[0], single_entry_shape[1], 
                                               single_entry_shape[2]),
                                        input_var=input_var)
    
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=480, filter_size=(3, 3),  #120
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
            W=lasagne.init.HeNormal())
    #network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))
    
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=240, filter_size=(2, 2), #120
            nonlinearity=lasagne.nonlinearities.leaky_rectify)
    
    network = lasagne.layers.DenseLayer(
           lasagne.layers.dropout(network, p=.5),
            num_units=120, #120*2*2
           nonlinearity=lasagne.nonlinearities.leaky_rectify)
    
    network = lasagne.layers.DenseLayer(
           lasagne.layers.dropout(network, p=.5),
            num_units=60,
           nonlinearity=lasagne.nonlinearities.leaky_rectify)
    
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=2,
            nonlinearity=lasagne.nonlinearities.softmax)

    return network

In [ ]:
num_epochs = 550
batchsize = 32
single_entry_shape = X_train.shape[1:]

# Prepare Theano variables for inputs and targets
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

network = build_cnn(single_entry_shape, input_var)

In [ ]:
# Create a loss expression for training, i.e., a scalar objective we want
# to minimize (for our multi-class problem, it is the cross-entropy loss):
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var) # multiclass_hinge_loss
loss = loss.mean()
# We could add some weight decay as well here, see lasagne.regularization.

# Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.001, momentum=0.9)

# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction, target_var)
test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                    dtype=theano.config.floatX)

# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    
    now = datetime.datetime.now()
    print("Epoch %d started on %s" % (epoch + 1, now.strftime("%Y-%m-%d %H:%M")))
    
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batchsize, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, batchsize, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

In [ ]:
# SAVE NETWORK
#np.savez('../model_needle_july1.npz', *lasagne.layers.get_all_param_values(network))

In [ ]:
# LOAD NETWORK
"""
with np.load('../model_needle_july1.npz') as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]
lasagne.layers.set_all_param_values(network, param_values)
"""

In [ ]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
predict_fn = theano.function([input_var], T.argmax(test_prediction, axis=1))
predict_confidence = theano.function([input_var], test_prediction)

In [ ]:
ind = 17
a = X_test[ind,:,:,:].reshape((1,3*((2*half_patch_size)+1),(2*half_patch_size)+1,(2*half_patch_size)+1))
predict_label_patch = predict_fn(a)[0]

print(predict_label_patch, y_test[ind])

In [ ]:
imshow(a[0,5,:,:], cmap='gray')

In [ ]:
#test_pat_sitk = sitk.ReadImage('LabelMaps/64/case.nrrd')
test_pat = get_resized_img('../LabelMaps/64/case.nrrd')
test_pat = test_pat[140:,15:180,80:230]
test_pat = pad_volume(test_pat, half_patch_size=half_patch_size)

pat_sitk = sitk.GetImageFromArray(test_pat.astype(np.float32))
sitk.WriteImage(pat_sitk, '../test_pat.nrrd')

final_label = np.zeros_like(test_pat)
test_pat.shape

In [ ]:
z_size, x_size, y_size = test_pat.shape

for z in xrange(half_patch_size, z_size-half_patch_size):
    print(z)
    for x in xrange(half_patch_size, x_size-half_patch_size):
        for y in xrange(half_patch_size, y_size-half_patch_size):
            patient_patch_img = test_pat[z-half_patch_size:z+half_patch_size+1,x-half_patch_size:x+half_patch_size+1,y-half_patch_size:y+half_patch_size+1]
            patient_patch = patchimg2differentview(patient_patch_img).reshape((1,3*((2*half_patch_size)+1),(2*half_patch_size)+1,(2*half_patch_size)+1)).astype(np.float32)
            patient_patch -= m
            patient_patch /= s
            predicted_label = int(predict_fn(patient_patch)[0])
            
            final_label[z,x,y] = predicted_label

final_label_sitk = sitk.GetImageFromArray(final_label)
sitk.WriteImage(final_label_sitk, '../test_label.nrrd')

In [ ]:
print("Done!")